In [9]:
from crawlbase import CrawlingAPI
from bs4 import BeautifulSoup
import pickle
import pandas as pd
import time
from openai import OpenAI
import os

def scrape(sub_red):
    token = 'dLN3qq4_CCZeNJTAPTwGdA'
    reddit_url = 'https://www.reddit.com/r/' + sub_red + '/new/'
    api = CrawlingAPI({'token': token})
    response = api.get(reddit_url)

    if response['status_code'] == 200:
        return response['body']
    else:
        print('Error')
# html_data = scrape('depression')

In [10]:
def process(doc):
    client = OpenAI(api_key = os.environ["OPENAI_API_KEY"])
    response = client.responses.create(
        model='gpt-5.1',
        input="is this post suicidal?" + " '" + doc + "' " + 'return True or False'
    )
    if 'True' in response.output_text:
        return True
    else: return False

In [16]:
def save_post(html_data,sub_red):
    soup = BeautifulSoup(html_data, 'html.parser')
    instances = soup.find_all('div', property="schema:articleBody")
    docs = []
    for instance in instances:
        p_tags = []
        for p_tag in instance:
            if p_tag:
                stripped = p_tag.get_text().strip()
                p_tags.append(stripped)
        docs.append(''.join(p_tags))

    with open('model.p', 'rb') as file:
        model = pickle.load(file)
    with open('vectorizer.p', 'rb') as file:
        vectorizer = pickle.load(file)

    

    verdicts = pd.read_csv('verdicts_both.csv')
    for doc in docs:
        bool_ai = process(doc)
        
        vectorized = vectorizer.transform([doc])
        predict_probas = model.predict_proba(vectorized)[:, 1]
        threshold = 0.9

        bool_m = (predict_probas >= threshold)

        if not (verdicts['post'] == doc).any() and doc:
            verdicts = pd.concat([verdicts, pd.DataFrame({'post':doc,'verdict_ai':bool_ai, 'verdict_m':bool_m, 'sub_red':sub_red},index=[0])])
            verdicts.to_csv('verdicts_both.csv', index=False)
            print('saved a post from r/' + sub_red + ' ' + str(bool_m) + str(bool_ai))
# save_post(html_data,'depression')



In [20]:
sub_reds = ['depression', 'sad', 'confessions']
# sub_reds = ['depression']
iterations = 0
# while iterations < 24:

for sub_red in sub_reds:
    try:

        save_post(scrape(sub_red),sub_red)

    except Exception as e: print(sub_red, e)

    # time.sleep(10)
# time.sleep(1800)

iterations += 1

saved a post from r/depression [ True]True
saved a post from r/depression [ True]True
saved a post from r/depression [False]False
saved a post from r/sad [False]False
saved a post from r/sad [ True]True
saved a post from r/sad [False]False
saved a post from r/confessions [False]False
saved a post from r/confessions [False]False
saved a post from r/confessions [False]False


In [19]:
verdicts = pd.DataFrame({'sub_red': [],'verdict_ai':[], 'verdict_m':[], 'post':[]})
verdicts.to_csv('verdicts_both.csv')